<img src="images/dask_horizontal.svg" align="right" width="30%">

# 懒惰执行

这里我们讨论了dask背后的一些概念，以及代码的懒惰执行。如果你急于继续学习本教程，你不需要通过这些材料，但它可能有助于理解dask的基本概念，这些东西如何与你可能已经使用的技术相结合，以及如何理解可能出错的事情。

## 引言

作为Python程序员，你可能已经执行了某些*技巧来实现大于内存的数据集的计算、并行执行或延迟/后台执行。也许这样的措辞，并不清楚我们的意思，但几个例子应该能让事情变得更清楚。Dask的意义在于让简单的事情变得简单，让复杂的事情变得可能!

除了[详细介绍](http://dask.pydata.org/en/latest/)之外，我们可以将Dask的基本原理总结为以下几点。

- 通过将数据分解成块并指定任务链，来处理不适合放在内存中的数据。
- 在集群的处理核心以及节点之间并行执行任务。
- 将计算移到数据上，而不是相反，以减少通信开销。

所有这些都让你能够最大限度地利用你的计算资源，但编程的方式是非常熟悉的：for-循环来构建基本任务，Python迭代器，以及NumPy(数组)和Pandas(数据框架)函数，分别用于多维或表格数据。

本笔记本的剩余部分将带你了解这些编程范式中的第一种。这比一些用户想要的更详细，他们可以跳过前面的迭代器、数组和数据框架部分；但会有一些数据处理任务不容易适合这些抽象的东西，需要回到这里的方法。

我们在笔记本的最后加入了一些例子，表明Dask如何构建背后的想法其实并没有那么新颖，有经验的程序员会在其他情况下遇到过部分设计。这些例子留给有兴趣的人。

## Dask是一个图执行引擎

Dask 允许您为您要进行的计算构建一个处方。这可能听起来很奇怪，但一个简单的例子将证明你可以在使用完全普通的Python函数和for-loops编程时实现这一点。我们在之前的笔记本中已经看到了这一点。

In [ ]:
from dask import delayed

@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y

在这里，我们使用了delayed注解来表明，我们希望这些函数的操作是懒惰的--保存输入集并仅在需求时执行。`dask.delayed`也是一个可以做到这一点的函数，不需要注释，保持原来的函数不变，例如。
```python
    delayed_inc = delayed(inc)
```

In [ ]:
# 貌似普通的代码
x = inc(15)
y = inc(30)
total = add(x, y)
# x、y和total都是延迟对象。
# 它们载有如何进行计算的规定。

调用延迟函数会创建一个延迟对象(`x, y, total`)，可以交互式地检查。制作这些对象在某种程度上等同于像`lambda`或函数包装器这样的结构（见下文）。每个对象都拥有一个描述任务图的简单字典，一个如何进行计算的完整规范。

我们可以将对象`total`所对应的计算链可视化如下；圆圈是函数，矩形是数据/结果。

In [ ]:
total.visualize()

但到目前为止，还没有任何函数被实际执行。这证明了Dask的图形创建部分（`delayed()`，在这个例子中）和Dask的图形执行部分之间的分工。

要在可视化中运行 "图"，并实际得到一个结果，可以这样做。

In [ ]:
# execute all tasks
total.compute()

**为什么要关心这个？

通过在执行任何事情之前建立一个我们要进行的计算规范，我们可以将该规范传递给一个*执行引擎*进行评估。在 Dask 的情况下，这个执行引擎可以在集群的许多节点上运行，因此您可以访问所有机器上的全部 CPU 核心和内存。Dask 将会智能地执行您的计算，并注意尽量减少内存中的数据量，同时对构成图形的任务进行并行处理。请注意，在下面的动画图中，四个工人正在处理（简单）图形，执行过程首先在分支上垂直进行，因此在进入新的分支之前，可以清除中间结果。

通过`delayed`和正常的python循环代码，可以建立非常复杂的图形，并传递给Dask执行。请看[模拟复杂ETL](https://blog.dask.org/2017/01/24/dask-custom)工作流的一个很好的例子。

![this](images/grid_search_schedule.gif)

### 练习

我们将把 "delayed "应用于一个真实的数据处理任务，尽管是一个简单的任务。

考虑用`pd.read_csv`读取三个CSV文件，然后测量它们的总长度。我们将考虑如何用普通的Python代码来做这件事，然后用delayed为这个过程建立一个图，最后用Dask执行这个图，方便的加速因子超过两个（只有三个输入要并行处理）。

In [ ]:
%run prep.py -d accounts

In [ ]:
import pandas as pd
import os
filenames = [os.path.join('data', 'accounts.%d.csv' % i) for i in [0, 1, 2]]
filenames

In [ ]:
%%time

# normal, sequential code
a = pd.read_csv(filenames[0])
b = pd.read_csv(filenames[1])
c = pd.read_csv(filenames[2])

na = len(a)
nb = len(b)
nc = len(c)

total = sum([na, nb, nc])
print(total)

你的任务是在原始Python代码上使用delayed函数重新创建这个图。你想要延迟的三个函数是`pd.read_csv`、`len`和`sum`...

```python
delayed_read_csv = delayed(pd.read_csv)
a = delayed_read_csv(filenames[0])
...

total = ...

# execute
%time total.compute()   
```

In [ ]:
# your verbose code here

接下来，使用循环重复这个过程，而不是写出所有的变量。

In [ ]:
# your concise code here

In [ ]:
## verbose version
delayed_read_csv = delayed(pd.read_csv)
a = delayed_read_csv(filenames[0])
b = delayed_read_csv(filenames[1])
c = delayed_read_csv(filenames[2])

delayed_len = delayed(len)
na = delayed_len(a)
nb = delayed_len(b)
nc = delayed_len(c)

delayed_sum = delayed(sum)

total = delayed_sum([na, nb, nc])
%time print(total.compute())


## concise version
csvs = [delayed(pd.read_csv)(fn) for fn in filenames]
lens = [delayed(len)(csv) for csv in csvs]
total = delayed(sum)(lens)
%time print(total.compute())


**注**：

延迟对象支持各种操作。
```python
    x2 = x + 1
```

如果`x`是一个延迟的结果（比如上面的`total`），那么`x2`也是如此。支持的操作包括算术运算符、项或分片选择、属性访问和方法调用--基本上任何可以表述为 "lambda "表达式的操作。

不支持的操作包括突变、setter方法、迭代（for）和bool（谓词）。

## 附录：进一步的细节和实例

下面的例子表明，Dask在处理大数据时，所做的各种事情与普通的Python编程并没有太大的区别。这些例子**只针对专家**，典型用户可以继续看教程中的下一个笔记本。

### 例1：简单单词计数

这个目录下有一个叫 "README.md "的文件。如何计算该文件中的字数？

最简单的方法是将所有的数据加载到内存中，在空白处进行分割，然后计算结果的数量。这里我们使用一个正则表达式来拆分单词。

In [ ]:
import re
splitter = re.compile('\w+')
with open('README.md', 'r') as f:
    data = f.read()
result = len(splitter.findall(data))
result

这种方法的问题是不具有扩展性--如果文件非常大，产生的单词列表可能会耗尽内存。我们可以很容易地避免这种情况，因为我们只需要一个简单的和，而且每一行完全独立于其他行。现在，我们评估每一个数据，并立即再次释放空间，所以我们可以在任意大的文件上执行。请注意，在时间效率和内存占用之间经常会有一个权衡：下面使用的内存非常少，但对于那些没有填满很大一部分内存的文件来说，可能会比较慢。一般情况下，人们希望文件块足够小，不会对内存造成压力，但又足够大，以便有效地使用CPU。

In [ ]:
result = 0
with open('README.md', 'r') as f:
    for line in f:
        result += len(splitter.findall(line))
result

### 例2：后台执行

有很多任务需要一段时间才能完成，但实际上并不需要耗费太多的CPU，例如任何需要通过网络通信的任务，或者需要用户输入的任务。在典型的顺序编程中，需要在进程完成时停止执行，然后继续执行。这对于用户体验来说是可怕的（想象一下缓慢的进度条，它锁定了应用程序，并且无法取消），并且浪费了时间（CPU在这期间本来可以做有用的工作）。

例如，我们可以启动进程，并得到它们的输出，如下所示:
```python
    import subprocess
    p = subprocess.Popen(command, stdout=subprocess.PIPE)
    p.returncode
```

任务是在一个单独的进程中运行的，返回代码将保持`None`，直到它完成，这时它将变为`0`。为了得到返回结果，我们需要`out = p.community()[0]`（如果进程没有完成，就会阻塞）。

同样，我们可以在后台启动Python进程和线程。有些方法允许在多个输入上进行映射，并收集结果，稍后会详细介绍。 线程启动，单元格立即完成，但与下载相关的数据要过一段时间才会出现在队列对象中。

In [ ]:
# 编辑sources.py来配置源位置
import sources
sources.lazy_url

In [ ]:
import threading
import queue
import urllib

def get_webdata(url, q):
    u = urllib.request.urlopen(url)
    # raise ValueError
    q.put(u.read())

q = queue.Queue()
t = threading.Thread(target=get_webdata, args=(sources.lazy_url, q))
t.start()

In [ ]:
# 取回结果到这个线程中。如果工作线程没有完成，这将会等待。
q.get()

考虑一下：如果在`get_webdata`函数中出现了异常，你会看到什么？你可以取消上面的 "raise "行，然后重新执行这两个单元。结果会怎样？有没有办法调试执行，找到错误的根本原因？

### 例3：延迟执行

在Python中，有很多方法可以指定你要执行的计算，但只能在*后运行它。

In [ ]:
def add(x, y):
    return x + y

# 有时，我们会用字符串来推迟计算
x = 15
y = 30
z = "add(x, y)"
eval(z)

In [ ]:
# 我们可以使用lambda或其他 "闭合 "的方式。
x = 15
y = 30
z = lambda: add(x, y)
z()

In [ ]:
# 类似的情况也发生在functools.partial中。

import functools
z = functools.partial(add, x, y)
z()

In [ ]:
# Python生成器默认延迟执行
# 许多Python函数都期望有这样的可迭代对象

def gen():
    res = x
    yield res
    res += y
    yield res

g = gen()

In [ ]:
# run once: we get one value and execution halts within the generator
# run again and the execution completes
next(g)

### Dask图

任何Dask对象，如上面的 "total"，都有一个属性，描述了产生该结果所需的计算。事实上，这正是我们一直在谈论的图，它可以被可视化。我们看到，这是一个简单的字典，其中键是唯一的任务标识，值是计算的函数和输入。

`delayed`是一个创建Dask图的方便机制，但喜欢冒险的人可能希望通过直接构建图字典来发挥充分的灵活性。详细信息可以在[这里](http://dask.pydata.org/en/latest/graphs.html)找到。

In [ ]:
total.dask

In [ ]:
dict(total.dask)